# Bucur Robert - Adrian
# Bogdan Gheorghe - Nicolae
## Grupa 10LF381

# Laborator 6

Versiunea 2020-04-01

## Modele de clasificare

Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 5; de exemplu, [ics.uci.edu](http://archive.ics.uci.edu/ml/datasets.php?format=mat&task=cla&att=&area=&numAtt=&numIns=&type=mvar&sort=nameUp&view=table). Cel putin doua seturi de date sa fie cu valori lipsa. 


1. (20 puncte) Aplicati o metoda de missing value imputation, unde este cazul; justificati si documentati metoda folosita.
1. (numar de modele * numar de seturi de date \* 1 punct = 20 de puncte) Pentru fiecare set de date aplicati 5 modele de clasificare din scikit learn. Pentru fiecare raportati: acuratete, scorul F1 - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare. Rularile se vor face cu valori fixate ale hiperparametrilor. 
1. (numar modele * 4 puncte = 20 puncte) Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Daca acelasi algoritm e folosit pentru mai multe seturi de date, puteti face o sectiune separata cu documentarea algoritmilor + trimitere la algoritm. 
1. (numar de modele * numar de seturi de date \* 1 punct = 20 de puncte) Pentru fiecare model: efectuati o cautare a hiperparametrilor optimi folosind grid search si random search (cu parametrul cv = 4), folosind 5 fold cross validation. 

Se acorda 20 de puncte din oficiu. 

Exemple de modele de clasificare:

1. [Multi-layer Perceptron classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)
1. [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)
1. [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
1. [Gaussian processes](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html#sklearn.gaussian_process.GaussianProcessClassifier)
1. [RBF](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html#sklearn.gaussian_process.kernels.RBF)
1. [Decision tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
1. [Random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)
1. [Gaussian Naive bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB) 

*Predare:*

1. Fiecare student va depune pe site-ul de elearning fisier Jupyter notebook sau arhiva cu astfel de fisiere; 
1. In fiecare fisier se specifica numele celor doi studenti care au lucra in echipa. 
1. Predarea se face in saptamana 13-17 aprilie 2020
1. Revedeti formele ulterioare ale acestui document pentru precizari despre: continutul rezultatelor raportate, modalitate de notare.

In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, f1_score

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

### Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 5; de exemplu, [ics.uci.edu](http://archive.ics.uci.edu/ml/datasets.php?format=mat&task=cla&att=&area=&numAtt=&numIns=&type=mvar&sort=nameUp&view=table). Cel putin doua seturi de date sa fie cu valori lipsa.

In [2]:
audit = pd.read_csv('data/audit/audit_data.csv', sep=';')
print('Audit shape:', audit.shape, '\n')

X_audit = audit.iloc[:, :-1]
y_audit = audit.iloc[:, -1]

print('NaN values:')
X_audit[X_audit.isnull().values]

Audit shape: (776, 17) 

NaN values:


,Sector_score,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score
642,55.57,0.23,2,0.0,2,0.23,5.0,2,NaN,2,2,0,2,0,2,2.0


In [3]:
banknote = pd.read_csv('data/banknote/banknote_data.csv', sep=';', header=None)
print('Banknote shape:', banknote.shape, '\n')

X_bank = banknote.iloc[:, :-1]
y_bank = banknote.iloc[:, -1]

print('NaN values:')
X_bank[X_bank.isnull().values]

Banknote shape: (1372, 5) 

NaN values:


,0,1,2,3


In [4]:
ionosphere = pd.read_csv('data/ionosphere/ionosphere_data.csv', sep=';', header=None)
print('Ionosphere shape:', ionosphere.shape, '\n')

X_ionosphere = ionosphere.iloc[:, :-1]
y_ionosphere = ionosphere.iloc[:, -1]

print('NaN values:')
X_ionosphere[X_ionosphere.isnull().values]

Ionosphere shape: (351, 35) 

NaN values:


,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33


In [5]:
breastcancer = pd.read_csv('data/breastCancer/breastCancer_data.csv', sep=',')
print('Breast cancer shape:', breastcancer.shape,'\n')

X_cancer = breastcancer.iloc[:, :-1]
y_cancer = breastcancer.iloc[:, -1]

print('NaN values:')
X_cancer[X_cancer.isnull().values]

Breast cancer shape: (116, 10) 

NaN values:


,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1
9,75,23.00,83,4.952,1.013839,NaN,11.57899,7.0913,318.302
115,86,27.18,138,19.910,6.777364,90.28,14.11000,4.3500,NaN


### Exercitiul 1 (20 puncte)
Aplicati o metoda de missing value imputation, unde este cazul; justificati si documentati metoda folosita.

In [6]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(X_audit)
X_audit = imp.transform(X_audit)

In [7]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(X_cancer)
X_cancer = imp.transform(X_cancer)

**Justificare: Am ales sa folosim o metoda de SimpleImputer, deoarece in seturile de date alese avem foarte putin valori lipsa si pentru a nu face drop la coloanele ce contin valori lipsa. Metoda SimpleImputer are un parametru strategy pe care l-am setat la 'most_frequent', deoarece valorile de pe coloanele cu valorile lipsa erau uniforme.**

**Documentare: Clasa SimpleImputer oferă strategii de bază pentru imputarea valorilor lipsă. Valorile care lipsesc pot fi imputate cu o valoare constantă furnizată sau folosind statisticile (media, mediana sau cele mai frecvente) ale fiecărei coloane în care sunt localizate valorile lipsă. Această clasă permite, de asemenea, diferite codări ale valorilor lipsă. SimpleImputer acceptă, de asemenea, date categorice reprezentate ca valori string sau categorii pandas atunci când se utilizează strategia 'most_frequent' sau 'constant'.**

### Exercitiul 2 (numar de modele * numar de seturi de date \* 1 punct = 20 de puncte)
Pentru fiecare set de date aplicati 5 modele de clasificare din scikit learn. Pentru fiecare raportati: acuratete, scorul F1 - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare. Rularile se vor face cu valori fixate ale hiperparametrilor.

# First dataset 'audit'
## First model used: DecisionTreeClassifier

In [8]:
dtc_audit = DecisionTreeClassifier(splitter='best', random_state=0, max_depth=1)

scores_dtc_audit = cross_validate(estimator=dtc_audit, scoring=('accuracy', 'f1'), X=X_audit, y=y_audit, cv=5, return_train_score=True)

print(sorted(scores_dtc_audit.keys()))
print()
print('Train accuracy :', scores_dtc_audit['train_accuracy'])
print('Test  accuracy :', scores_dtc_audit['test_accuracy'])
print('Train f1       :', scores_dtc_audit['train_f1'])
print('Test  f1       :', scores_dtc_audit['test_f1'])
print()
print('Mean train accuaracy :', scores_dtc_audit['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_dtc_audit['test_accuracy'].mean())
print('Mean train f1        :', scores_dtc_audit['train_f1'].mean())
print('Mean test  f1        :', scores_dtc_audit['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [1. 1. 1. 1. 1.]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [1. 1. 1. 1. 1.]

Mean train accuaracy : 1.0
Mean test  accuaracy : 1.0
Mean train f1        : 1.0
Mean test  f1        : 1.0


# First dataset 'audit'
## Second model used: KNeighborsClassifier

In [9]:
knn_audit = KNeighborsClassifier(n_neighbors=5, p=2)

scores_knn_audit = cross_validate(estimator=knn_audit, scoring=('accuracy', 'f1'), X=X_audit, y=y_audit, cv=5, return_train_score=True)

print(sorted(scores_knn_audit.keys()))
print()
print('Train accuracy :', scores_knn_audit['train_accuracy'])
print('Test  accuracy :', scores_knn_audit['test_accuracy'])
print('Train f1       :', scores_knn_audit['train_f1'])
print('Test  f1       :', scores_knn_audit['test_f1'])
print()
print('Mean train accuaracy :', scores_knn_audit['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_knn_audit['test_accuracy'].mean())
print('Mean train f1        :', scores_knn_audit['train_f1'].mean())
print('Mean test  f1        :', scores_knn_audit['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [0.99032258 0.98550725 0.98550725 0.98711755 0.99194847]
Test  accuracy : [0.97435897 0.99354839 0.79354839 0.99354839 0.40645161]
Train f1       : [0.99222798 0.98835705 0.98832685 0.98969072 0.99358151]
Test  f1       : [0.98       0.99487179 0.85454545 0.99481865 0.09803922]

Mean train accuaracy : 0.9880806191886137
Mean test  accuaracy : 0.8322911497105047
Mean train f1        : 0.9904368228776841
Mean test  f1        : 0.784455023590653


# First dataset 'audit'
## Third model used: SVM.SVC

In [10]:
svc_audit = SVC(C=1.0, gamma='auto')

scores_svc_audit = cross_validate(estimator=svc_audit, scoring=('accuracy', 'f1'), X=X_audit, y=y_audit, cv=5, return_train_score=True)

print(sorted(scores_svc_audit.keys()))
print()
print('Train accuracy :', scores_svc_audit['train_accuracy'])
print('Test  accuracy :', scores_svc_audit['test_accuracy'])
print('Train f1       :', scores_svc_audit['train_f1'])
print('Test  f1       :', scores_svc_audit['test_f1'])
print()
print('Mean train accuaracy :', scores_svc_audit['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_svc_audit['test_accuracy'].mean())
print('Mean train f1        :', scores_svc_audit['train_f1'].mean())
print('Mean test  f1        :', scores_svc_audit['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1.         1.         1.         0.99838969 1.        ]
Test  accuracy : [0.96794872 0.99354839 0.81290323 0.96774194 0.65806452]
Train f1       : [1.       1.       1.       0.998713 1.      ]
Test  f1       : [0.97512438 0.99487179 0.86995516 0.97354497 0.62411348]

Mean train accuaracy : 0.9996779388083736
Mean test  accuaracy : 0.8800413564929693
Mean train f1        : 0.9997425997425997
Mean test  f1        : 0.8875219557308398


# First dataset 'audit'
## Fourth model used: GaussianProcessClassifier

In [11]:
gpc_audit = GaussianProcessClassifier(random_state=0)

scores_gpc_audit = cross_validate(estimator=gpc_audit, scoring=('accuracy', 'f1'), X=X_audit, y=y_audit, cv=5, return_train_score=True)

print(sorted(scores_gpc_audit.keys()))
print()
print('Train accuracy :', scores_gpc_audit['train_accuracy'])
print('Test  accuracy :', scores_gpc_audit['test_accuracy'])
print('Train f1       :', scores_gpc_audit['train_f1'])
print('Test  f1       :', scores_gpc_audit['test_f1'])
print()
print('Mean train accuracy :', scores_gpc_audit['train_accuracy'].mean())
print('Mean test  accuracy :', scores_gpc_audit['test_accuracy'].mean())
print('Mean train f1       :', scores_gpc_audit['train_f1'].mean())
print('Mean test f1        :', scores_gpc_audit['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [0.95512821 0.95483871 0.79354839 0.98709677 0.37419355]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [0.96446701 0.96296296 0.85454545 0.98958333 0.        ]

Mean train accuracy : 1.0
Mean test  accuracy : 0.8129611248966088
Mean train f1       : 1.0
Mean test f1        : 0.7543117511835785


# First dataset 'audit'
## Fifth model used: RandomForestClassifier

In [12]:
rcf_audit = RandomForestClassifier(max_depth=2, random_state=0)

scores_rcf_audit = cross_validate(estimator=rcf_audit, scoring=('accuracy', 'f1'), X=X_audit, y=y_audit, cv=5, return_train_score=True)

print(sorted(scores_rcf_audit.keys()))
print()
print('Train accuracy      :', scores_rcf_audit['train_accuracy'])
print('Test  accuracy      :', scores_rcf_audit['test_accuracy'])
print('Train f1            :', scores_rcf_audit['train_f1'])
print('Test  f1            :', scores_rcf_audit['test_f1'])
print()
print('Mean train accuracy :', scores_rcf_audit['train_accuracy'].mean())
print('Mean test  accuracy :', scores_rcf_audit['test_accuracy'].mean())
print('Mean train f1       :', scores_rcf_audit['train_f1'].mean())
print('Mean test f1        :', scores_rcf_audit['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [1. 1. 1. 1. 1.]
Test  accuracy      : [1.         1.         1.         1.         0.97419355]
Train f1            : [1. 1. 1. 1. 1.]
Test  f1            : [1.         1.         1.         1.         0.97894737]

Mean train accuracy : 1.0
Mean test  accuracy : 0.9948387096774194
Mean train f1       : 1.0
Mean test f1        : 0.9957894736842106


# Second dataset 'banknote'
## First model used: DecisionTreeClassifier

In [13]:
dtc_bank = DecisionTreeClassifier(splitter='random', random_state=0)

scores_dtc_bank = cross_validate(estimator=dtc_bank, scoring=('accuracy', 'f1'), X=X_bank, y=y_bank, cv=5, return_train_score=True)

print(sorted(scores_dtc_bank.keys()))
print()
print('Train accuracy :', scores_dtc_bank['train_accuracy'])
print('Test  accuracy :', scores_dtc_bank['test_accuracy'])
print('Train f1       :', scores_dtc_bank['train_f1'])
print('Test  f1       :', scores_dtc_bank['test_f1'])
print()
print('Mean train accuaracy :', scores_dtc_bank['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_dtc_bank['test_accuracy'].mean())
print('Mean train f1        :', scores_dtc_bank['train_f1'].mean())
print('Mean test  f1        :', scores_dtc_bank['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [0.99272727 0.98545455 0.98540146 0.99635036 0.97445255]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [0.99186992 0.98373984 0.98373984 0.99591837 0.97142857]

Mean train accuaracy : 1.0
Mean test  accuaracy : 0.9868772395487723
Mean train f1        : 1.0
Mean test  f1        : 0.985339306454289


# Second dataset 'banknote'
## Second model used: KNeighborsClassifier

In [14]:
knn_bank = KNeighborsClassifier(n_neighbors=5, p=2)

scores_knn_bank = cross_validate(estimator=knn_bank, scoring=('accuracy', 'f1'), X=X_bank, y=y_bank, cv=5, return_train_score=True)

print(sorted(scores_knn_bank.keys()))
print()
print('Train accuracy :', scores_knn_bank['train_accuracy'])
print('Test  accuracy :', scores_knn_bank['test_accuracy'])
print('Train f1       :', scores_knn_bank['train_f1'])
print('Test  f1       :', scores_knn_bank['test_f1'])
print()
print('Mean train accuaracy :', scores_knn_bank['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_knn_bank['test_accuracy'].mean())
print('Mean train f1        :', scores_knn_bank['train_f1'].mean())
print('Mean test  f1        :', scores_knn_bank['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [0.99544211 0.99453054 0.99635701 0.99453552 0.99726776]
Test  accuracy : [0.99636364 1.         0.99270073 1.         0.98905109]
Train f1       : [0.99488229 0.99386503 0.99591837 0.99386503 0.99692308]
Test  f1       : [0.99591837 1.         0.99173554 1.         0.98785425]

Mean train accuaracy : 0.9956265888256265
Mean test  accuaracy : 0.9956230922362309
Mean train f1        : 0.9950907596705129
Mean test  f1        : 0.9951016311098334


# Second dataset 'banknote'
## Third model used: SVM.SVC

In [15]:
svc_bank = SVC(C=1.0, gamma='auto')

scores_svc_bank = cross_validate(estimator=svc_bank, scoring=('accuracy', 'f1'), X=X_bank, y=y_bank, cv=5, return_train_score=True)

print(sorted(scores_svc_bank.keys()))
print()
print('Train accuracy :', scores_svc_bank['train_accuracy'])
print('Test  accuracy :', scores_svc_bank['test_accuracy'])
print('Train f1       :', scores_svc_bank['train_f1'])
print('Test  f1       :', scores_svc_bank['test_f1'])
print()
print('Mean train accuaracy :', scores_svc_bank['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_svc_bank['test_accuracy'].mean())
print('Mean train f1        :', scores_svc_bank['train_f1'].mean())
print('Mean test  f1        :', scores_svc_bank['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [1.         0.99636364 0.99270073 1.         1.        ]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [1.         0.99588477 0.99173554 1.         1.        ]

Mean train accuaracy : 1.0
Mean test  accuaracy : 0.9978128732581286
Mean train f1        : 1.0
Mean test  f1        : 0.9975240621705268


# Second dataset 'banknote'
## Fourth model used: GaussianProcessClassifier

In [16]:
gpc_bank = GaussianProcessClassifier(random_state=0)

scores_gpc_bank = cross_validate(estimator=gpc_bank, scoring=('accuracy', 'f1'), X=X_bank, y=y_bank, cv=5, return_train_score=True)

print(sorted(scores_gpc_bank.keys()))
print()
print('Train accuracy :', scores_gpc_bank['train_accuracy'])
print('Test  accuracy :', scores_gpc_bank['test_accuracy'])
print('Train f1       :', scores_gpc_bank['train_f1'])
print('Test  f1       :', scores_gpc_bank['test_f1'])
print()
print('Mean train accuracy :', scores_gpc_bank['train_accuracy'].mean())
print('Mean test  accuracy :', scores_gpc_bank['test_accuracy'].mean())
print('Mean train f1       :', scores_gpc_bank['train_f1'].mean())
print('Mean test f1        :', scores_gpc_bank['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [1.         0.99636364 0.99270073 0.99635036 1.        ]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [1.         0.99588477 0.99173554 0.99591837 1.        ]

Mean train accuracy : 1.0
Mean test  accuracy : 0.9970829462508295
Mean train f1       : 1.0
Mean test f1        : 0.9967077356399144


# Second dataset 'banknote'
## Fifth model used: RandomForestClassifier

In [17]:
rcf_bank = RandomForestClassifier(max_depth=1, random_state=0)

scores_rcf_bank = cross_validate(estimator=rcf_bank, scoring=('accuracy', 'f1'), X=X_bank, y=y_bank, cv=5, return_train_score=True)

print(sorted(scores_rcf_bank.keys()))
print()
print('Train accuracy      :', scores_rcf_bank['train_accuracy'])
print('Test  accuracy      :', scores_rcf_bank['test_accuracy'])
print('Train f1            :', scores_rcf_bank['train_f1'])
print('Test  f1            :', scores_rcf_bank['test_f1'])
print()
print('Mean train accuracy :', scores_rcf_bank['train_accuracy'].mean())
print('Mean test  accuracy :', scores_rcf_bank['test_accuracy'].mean())
print('Mean train f1       :', scores_rcf_bank['train_f1'].mean())
print('Mean test f1        :', scores_rcf_bank['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [0.89243391 0.89881495 0.89617486 0.89162113 0.89435337]
Test  accuracy      : [0.90181818 0.87272727 0.88686131 0.90875912 0.89781022]
Train f1            : [0.8691796  0.87734807 0.87417219 0.86792453 0.87168142]
Test  f1            : [0.88105727 0.84304933 0.86098655 0.89082969 0.87610619]

Mean train accuracy : 0.8946796446011891
Mean test  accuracy : 0.8935952222959521
Mean train f1       : 0.8720611593693629
Mean test f1        : 0.8704058064350676


# Third dataset 'breastCancer'
## First model used: DecisionTreeClassifier

In [18]:
dtc_cancer = DecisionTreeClassifier(splitter='random', random_state=0)

scores_dtc_cancer = cross_validate(estimator=dtc_cancer, scoring=('accuracy','f1'), X=X_cancer, y=y_cancer, cv=5, return_train_score=True)

print(sorted(scores_dtc_cancer.keys()))
print()
print('Train accuracy :', scores_dtc_cancer['train_accuracy'])
print('Test  accuracy :', scores_dtc_cancer['test_accuracy'])
print('Train f1       :', scores_dtc_cancer['train_f1'])
print('Test  f1       :', scores_dtc_cancer['test_f1'])
print()
print('Mean train accuaracy :', scores_dtc_cancer['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_dtc_cancer['test_accuracy'].mean())
print('Mean train f1        :', scores_dtc_cancer['train_f1'].mean())
print('Mean test  f1        :', scores_dtc_cancer['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [0.58333333 0.65217391 0.65217391 0.65217391 0.69565217]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [0.54545455 0.63636364 0.5        0.55555556 0.74074074]

Mean train accuaracy : 1.0
Mean test  accuaracy : 0.6471014492753623
Mean train f1        : 1.0
Mean test  f1        : 0.5956228956228955


# Third dataset 'breastCancer'
## Second model used: KNeighborsClassifier

In [19]:
knn_cancer = KNeighborsClassifier(n_neighbors=5, p=2)

scores_knn_cancer = cross_validate(estimator=knn_cancer, scoring=('accuracy','f1'), X=X_cancer, y=y_cancer, cv=5, return_train_score=True)

print(sorted(scores_knn_cancer.keys()))
print()
print('Train accuracy      :', scores_knn_cancer['train_accuracy'])
print('Test  accuracy      :', scores_knn_cancer['test_accuracy'])
print('Train f1            :', scores_knn_cancer['train_f1'])
print('Test  f1            :', scores_knn_cancer['test_f1'])
print()
print('Mean train accuracy :', scores_knn_cancer['train_accuracy'].mean())
print('Mean test  accuracy :', scores_knn_cancer['test_accuracy'].mean())
print('Mean train f1       :', scores_knn_cancer['train_f1'].mean())
print('Mean test f1        :', scores_knn_cancer['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [0.76086957 0.70967742 0.67741935 0.72043011 0.70967742]
Test  accuracy      : [0.45833333 0.65217391 0.47826087 0.47826087 0.43478261]
Train f1            : [0.71052632 0.65822785 0.61538462 0.675      0.64935065]
Test  f1            : [0.48       0.66666667 0.4        0.33333333 0.13333333]

Mean train accuracy : 0.7156147732585321
Mean test  accuracy : 0.5003623188405797
Mean train f1       : 0.6616978857252008
Mean test f1        : 0.40266666666666673


# Second dataset 'breastCancer'
## Third model used:  SVM.SVC

In [20]:
svc_cancer = SVC(gamma='auto')

scores_svc_cancer = cross_validate(estimator=svc_cancer, scoring=('accuracy','f1'), X=X_cancer, y=y_cancer, cv=5, return_train_score=True)

print(sorted(scores_svc_cancer.keys()))
print()
print('Train accuracy      :', scores_svc_cancer['train_accuracy'])
print('Test  accuracy      :', scores_svc_cancer['test_accuracy'])
print('Train f1            :', scores_svc_cancer['train_f1'])
print('Test  f1            :', scores_svc_cancer['test_f1'])
print()
print('Mean train accuracy :', scores_svc_cancer['train_accuracy'].mean())
print('Mean test  accuracy :', scores_svc_cancer['test_accuracy'].mean())
print('Mean train f1       :', scores_svc_cancer['train_f1'].mean())
print('Mean test f1        :', scores_svc_cancer['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [1. 1. 1. 1. 1.]
Test  accuracy      : [0.54166667 0.52173913 0.56521739 0.56521739 0.56521739]
Train f1            : [1. 1. 1. 1. 1.]
Test  f1            : [0. 0. 0. 0. 0.]

Mean train accuracy : 1.0
Mean test  accuracy : 0.5518115942028985
Mean train f1       : 1.0
Mean test f1        : 0.0


# Third dataset 'breastCancer'
## Fourth model used: GaussianProcessClassifier

In [21]:
gpc_cancer = GaussianProcessClassifier(random_state=0)

scores_gpc_cancer = cross_validate(estimator=gpc_cancer, scoring=('accuracy','f1'), X=X_cancer, y=y_cancer, cv=5, return_train_score=True)

print(sorted(scores_gpc_cancer.keys()))
print()
print('Train accuracy :', scores_gpc_cancer['train_accuracy'])
print('Test  accuracy :', scores_gpc_cancer['test_accuracy'])
print('Train f1       :', scores_gpc_cancer['train_f1'])
print('Test  f1       :', scores_gpc_cancer['test_f1'])
print()
print('Mean train accuracy :', scores_gpc_cancer['train_accuracy'].mean())
print('Mean test  accuracy :', scores_gpc_cancer['test_accuracy'].mean())
print('Mean train f1       :', scores_gpc_cancer['train_f1'].mean())
print('Mean test f1        :', scores_gpc_cancer['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [0.58333333 0.47826087 0.43478261 0.39130435 0.43478261]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [0.61538462 0.53846154 0.51851852 0.36363636 0.51851852]

Mean train accuracy : 1.0
Mean test  accuracy : 0.46449275362318837
Mean train f1       : 1.0
Mean test f1        : 0.510903910903911


# Third dataset 'breastCancer'
## Fifth model used: RandomForestClassifier

In [22]:
rcf_cancer = RandomForestClassifier(max_depth=2, random_state=0)

scores_rcf_cancer = cross_validate(estimator=rcf_cancer, scoring=('accuracy','f1'), X=X_cancer, y=y_cancer, cv=5, return_train_score=True)

print(sorted(scores_rcf_cancer.keys()))
print()
print('Train accuracy      :', scores_rcf_cancer['train_accuracy'])
print('Test  accuracy      :', scores_rcf_cancer['test_accuracy'])
print('Train f1            :', scores_rcf_cancer['train_f1'])
print('Test  f1            :', scores_rcf_cancer['test_f1'])
print()
print('Mean train accuracy :', scores_rcf_cancer['train_accuracy'].mean())
print('Mean test  accuracy :', scores_rcf_cancer['test_accuracy'].mean())
print('Mean train f1       :', scores_rcf_cancer['train_f1'].mean())
print('Mean test f1        :', scores_rcf_cancer['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [0.82608696 0.84946237 0.84946237 0.91397849 0.92473118]
Test  accuracy      : [0.58333333 0.82608696 0.7826087  0.65217391 0.47826087]
Train f1            : [0.78947368 0.83333333 0.81081081 0.9        0.91358025]
Test  f1            : [0.44444444 0.83333333 0.66666667 0.55555556 0.45454545]

Mean train accuracy : 0.8727442730247779
Mean test  accuracy : 0.6644927536231884
Mean train f1       : 0.8494396150536501
Mean test f1        : 0.5909090909090909


# Fourth dataset 'ionosphere'
## First model used: DecisionTreeClassifier

In [23]:
dtc_ionosphere = DecisionTreeClassifier(splitter='random', random_state=0)

scores_dtc_ionosphere = cross_validate(estimator=dtc_ionosphere, scoring=('accuracy','f1'), X=X_ionosphere, y=y_ionosphere, cv=5, return_train_score=True)

print(sorted(scores_dtc_ionosphere.keys()))
print()
print('Train accuracy :', scores_dtc_ionosphere['train_accuracy'])
print('Test  accuracy :', scores_dtc_ionosphere['test_accuracy'])
print('Train f1       :', scores_dtc_ionosphere['train_f1'])
print('Test  f1       :', scores_dtc_ionosphere['test_f1'])
print()
print('Mean train accuaracy :', scores_dtc_ionosphere['train_accuracy'].mean())
print('Mean test  accuaracy :', scores_dtc_ionosphere['test_accuracy'].mean())
print('Mean train f1        :', scores_dtc_ionosphere['train_f1'].mean())
print('Mean test  f1        :', scores_dtc_ionosphere['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [1. 1. 1. 1. 1.]
Test  accuracy : [0.88732394 0.91428571 0.87142857 0.92857143 0.95714286]
Train f1       : [1. 1. 1. 1. 1.]
Test  f1       : [0.91304348 0.93181818 0.9010989  0.94382022 0.96703297]

Mean train accuaracy : 1.0
Mean test  accuaracy : 0.9117505030181088
Mean train f1        : 1.0
Mean test  f1        : 0.931362750586004


# Fourth dataset 'ionosphere'
## Second model used: KNeighborsClassifier

In [24]:
knn_ionosphere = KNeighborsClassifier(n_neighbors=5, p=2)

scores_knn_ionosphere = cross_validate(estimator=knn_ionosphere, scoring=('accuracy','f1'), X=X_ionosphere, y=y_ionosphere, cv=5, return_train_score=True)

print(sorted(scores_knn_ionosphere.keys()))
print()
print('Train accuracy      :', scores_knn_ionosphere['train_accuracy'])
print('Test  accuracy      :', scores_knn_ionosphere['test_accuracy'])
print('Train f1            :', scores_knn_ionosphere['train_f1'])
print('Test  f1            :', scores_knn_ionosphere['test_f1'])
print()
print('Mean train accuracy :', scores_knn_ionosphere['train_accuracy'].mean())
print('Mean test  accuracy :', scores_knn_ionosphere['test_accuracy'].mean())
print('Mean train f1       :', scores_knn_ionosphere['train_f1'].mean())
print('Mean test f1        :', scores_knn_ionosphere['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [0.87857143 0.90035587 0.86120996 0.85409253 0.87188612]
Test  accuracy      : [0.83098592 0.77142857 0.8        0.88571429 0.84285714]
Train f1            : [0.91237113 0.92670157 0.90076336 0.89672544 0.90816327]
Test  f1            : [0.88235294 0.84313725 0.8627451  0.91666667 0.88888889]

Mean train accuracy : 0.8732231825114386
Mean test  accuracy : 0.8261971830985916
Mean train f1       : 0.9089449539184082
Mean test f1        : 0.8787581699346404


# Fourth dataset 'ionosphere'
## Third model used: SVM.SVC

In [25]:
svc_ionosphere = SVC(gamma='auto')

scores_svc_ionosphere = cross_validate(estimator=svc_ionosphere, scoring=('accuracy','f1'), X=X_ionosphere, y=y_ionosphere, cv=5, return_train_score=True)

print(sorted(scores_svc_ionosphere.keys()))
print()
print('Train accuracy      :', scores_svc_ionosphere['train_accuracy'])
print('Test  accuracy      :', scores_svc_ionosphere['test_accuracy'])
print('Train f1            :', scores_svc_ionosphere['train_f1'])
print('Test  f1            :', scores_svc_ionosphere['test_f1'])
print()
print('Mean train accuracy :', scores_svc_ionosphere['train_accuracy'].mean())
print('Mean test  accuracy :', scores_svc_ionosphere['test_accuracy'].mean())
print('Mean train f1       :', scores_svc_ionosphere['train_f1'].mean())
print('Mean test f1        :', scores_svc_ionosphere['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [0.94642857 0.95373665 0.95017794 0.93950178 0.94661922]
Test  accuracy      : [0.94366197 0.85714286 0.88571429 0.98571429 0.92857143]
Train f1            : [0.95978552 0.96476965 0.96216216 0.95442359 0.95978552]
Test  f1            : [0.95744681 0.89795918 0.91836735 0.98901099 0.94623656]

Mean train accuracy : 0.9472928317234368
Mean test  accuracy : 0.9201609657947687
Mean train f1       : 0.9601852895856687
Mean test f1        : 0.9418041774547316


# Fourth dataset 'ionosphere'
## Fourth model used: GaussianProcessClassifier

In [26]:
gpc_ionosphere = GaussianProcessClassifier(random_state=0)

scores_gpc_ionosphere = cross_validate(estimator=gpc_ionosphere, scoring=('accuracy','f1'), X=X_ionosphere, y=y_ionosphere, cv=5, return_train_score=True)

print(sorted(scores_gpc_ionosphere.keys()))
print()
print('Train accuracy :', scores_gpc_ionosphere['train_accuracy'])
print('Test  accuracy :', scores_gpc_ionosphere['test_accuracy'])
print('Train f1       :', scores_gpc_ionosphere['train_f1'])
print('Test  f1       :', scores_gpc_ionosphere['test_f1'])
print()
print('Mean train accuracy :', scores_gpc_ionosphere['train_accuracy'].mean())
print('Mean test  accuracy :', scores_gpc_ionosphere['test_accuracy'].mean())
print('Mean train f1       :', scores_gpc_ionosphere['train_f1'].mean())
print('Mean test f1        :', scores_gpc_ionosphere['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy : [0.96071429 0.97864769 0.97508897 0.96797153 0.96797153]
Test  accuracy : [0.84507042 0.78571429 0.81428571 0.92857143 0.87142857]
Train f1       : [0.96986301 0.98342541 0.98071625 0.97547684 0.97534247]
Test  f1       : [0.89108911 0.85148515 0.87378641 0.94623656 0.90721649]

Mean train accuracy : 0.9700788002033554
Mean test  accuracy : 0.8490140845070424
Mean train f1       : 0.9769647972994558
Mean test f1        : 0.8939627438355757


# Fourth dataset 'ionosphere'
## Fifth model used: RandomForestClassifier

In [27]:
rcf_ionosphere = RandomForestClassifier(max_depth=2, random_state=0)

scores_rcf_ionosphere = cross_validate(estimator=rcf_ionosphere, scoring=('accuracy','f1'), X=X_ionosphere, y=y_ionosphere, cv=5, return_train_score=True)

print(sorted(scores_rcf_ionosphere.keys()))
print()
print('Train accuracy      :', scores_rcf_ionosphere['train_accuracy'])
print('Test  accuracy      :', scores_rcf_ionosphere['test_accuracy'])
print('Train f1            :', scores_rcf_ionosphere['train_f1'])
print('Test  f1            :', scores_rcf_ionosphere['test_f1'])
print()
print('Mean train accuracy :', scores_rcf_ionosphere['train_accuracy'].mean())
print('Mean test  accuracy :', scores_rcf_ionosphere['test_accuracy'].mean())
print('Mean train f1       :', scores_rcf_ionosphere['train_f1'].mean())
print('Mean test f1        :', scores_rcf_ionosphere['test_f1'].mean())

['fit_time', 'score_time', 'test_accuracy', 'test_f1', 'train_accuracy', 'train_f1']

Train accuracy      : [0.925      0.93950178 0.95017794 0.91814947 0.91814947]
Test  accuracy      : [0.90140845 0.85714286 0.87142857 0.97142857 0.94285714]
Train f1            : [0.94459103 0.95466667 0.96256684 0.93963255 0.93994778]
Test  f1            : [0.92783505 0.89795918 0.90909091 0.97826087 0.95652174]

Mean train accuracy : 0.9301957295373665
Mean test  accuracy : 0.9088531187122737
Mean train f1       : 0.9482809734441618
Mean test f1        : 0.9339335506012845


### Exercitiul 3 (numar modele * 4 puncte = 20 puncte)
Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Daca acelasi algoritm e folosit pentru mai multe seturi de date, puteti face o sectiune separata cu documentarea algoritmilor + trimitere la algoritm. 

## First model DecisionTreeClassifier
**Arborii de decizie sunt o metodă de învățare supravegheată non-parametrică folosită pentru clasificare și regresie. Scopul este de a crea un model care prezice valoarea unei variabile țintă, învățând reguli simple de decizie deduse din caracteristicile datelor.**

**Avantaje ale arborilor de decizie:**
- Simplu de înțeles și de interpretat. Arborii pot fi vizualizați.
- Necesită puțină pregătire a datelor. Alte tehnici necesită adesea normalizarea datelor, trebuie create variabile manechin și eliminarea valorilor necompletate. De reținut însă că acest modul nu acceptă valori lipsă.
- Costul utilizării arborelui (adică, prezicerea datelor) este logaritmic în numărul de puncte de date utilizate pentru antrenarea arborelui.
- Capabil să se ocupe de date atât numerice cât și categorice. Alte tehnici sunt de obicei specializate în analiza seturilor de date care au un singur tip de variabilă.
- Capabil să gestioneze problemele cu ieșire multiplă.
- Utilizează un model de 'cutie albă'. Dacă o anumită situație este observabilă într-un model, explicația pentru această condiție este ușor explicată prin logica booleană. În schimb, într-un model de cutie neagră (de exemplu, într-o rețea neuronală artificială), rezultatele pot fi mai dificil de interpretat.
- Este posibil să se valideze un model folosind teste statistice. Acest lucru face posibilă contabilizarea fiabilității modelului.
- Funcționează bine chiar dacă presupunerile sale sunt oarecum încălcate de modelul adevărat din care au fost generate datele.

**Dezavantajele arborilor de decizie:**
- Cei care învață arbori de decizie pot crea arbori supra-complecși care nu generalizează bine datele. Aceasta se numește supra-montare. Pentru a evita această problemă, sunt necesare mecanisme precum tăierea, stabilirea numărului minim de eșantioane necesare la un nod de frunze sau setarea adâncimii maxime a arborelui.
- Arborii de decizii pot fi instabili, deoarece mici variații ale datelor pot duce la generarea unui arbore complet diferit. Această problemă este atenuată folosind arbori de decizie în cadrul unui ansamblu.
- Problema învățării unui arbore decizional optim este cunoscută a fi NP-completă sub mai multe aspecte ale optimității și chiar pentru concepte simple. În consecință, algoritmii de învățare practică pe arbori de decizie se bazează pe algoritmi euristici, cum ar fi algoritmul lacom în care se iau decizii locale optime la fiecare nod. Astfel de algoritmi nu pot garanta returnarea arborelui decizional optim. Acest lucru poate fi atenuat prin antrenarea mai multor copaci într-un cursant de ansamblu, unde caracteristicile și eșantioanele sunt eșantionate aleatoriu cu înlocuire.
- Există concepte greu de învățat, deoarece arborii de decizie nu le exprimă ușor, cum ar fi XOR, probleme de paritate sau multiplexor.
- Cei care învață arborii decizionali creează arbori părtinitori dacă domină anumite clase. Prin urmare, se recomandă echilibrarea setului de date înainte de a se potrivi cu arborele de decizie.

## Second model KNeighborsClassifier
### Despre: 
Clasificarea bazată pe KNeighborsClassifier este un tip de învățare bazată pe instanțe care nu generalizează: nu încearcă să construiască un model intern general, ci pur și simplu stochează cazuri de instruire. Clasificarea se calculează pe baza votului cu majoritate simplă a vecinilor apropiați din fiecare punct: un semn de întrebare este atribuit clasei de date care are cei mai mulți reprezentanți din vecinii apropiați ai acestui punct.

Scikit-learning implementează două clasificatoare diferite de vecini: KNeighborsClassifier implementează învățarea bazată pe
vecinii cei mai apropiați ai fiecărui punct de interogare, unde este o valoare integrală specificată de utilizator. RadiusNeighborsClassifier implementează învățarea în funcție de numărul de vecini pe o rază fixă ​​a fiecărui punct de antrenament, unde este o valoare în virgulă flotantă specificată de utilizator.

Clasificarea vecinătății în KNeighborsClassifier este cea mai frecvent utilizată tehnică. Alegerea optimă a valorii depinde în mare măsură de date: în general una mai mare suprima efectele zgomotului, dar face ca limitele de clasificare să fie mai puțin distincte.

### Parametrii:
- n_neighbors : int, optional (default = 5) - Numărul de vecini care vor fi folosiți în mod implicit.
- weights : str, optional (default = ‘uniform’) - funcția de greutate folosită în predicție. Valori posibile: „Uniform”, „Distanță”,  callable.
- algorithm : {‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, optional - Algoritmul folosit pentru calcularea vecinilor apropiați.
- leaf_size : int, optional (default = 30) - Dimensiunea frunzelor a trecut la BallTree sau KDTree. Acest lucru poate afecta viteza de construcție și interogare, precum și memoria necesară pentru a păstra arborele. Valoarea optimă depinde de natura problemei.
- p : integer, optional (default = 2) - M distanței de utilizat pentru arbore. Metrica implicită este minkowski, iar cu p = 2 este echivalentă cu metrica euclidiană standard.
- metric : string, default ‘minkowski’ - Pe măsură ce leaf_size crește, memoria necesară pentru a stoca o structură de arbore scade. Acest lucru este important mai ales în cazul arborelui cu bilă, care stochează un centroid dimensional pentru fiecare nod.
- metric_params : dict, optional (default = None) - Argumente suplimentare de cuvinte cheie pentru funcția metrică.
- n_jobs : int or None, optional (default=None) - Numărul de locuri de muncă paralele care trebuie executate pentru căutarea vecinilor.

### Atribute:
- classes_ : array of shape (n_classes,) - Etichete de clasă cunoscute de clasificator.
- effective_metric_ : string or callble - Măsura distanței utilizate. Acesta va fi același cu parametrul metric sau un sinonim cu acesta, de ex. „Euclidian” dacă parametrul metric este setat pe „minkowski” și parametrul p setat pe 2.
- effective_metric_params_dict - Argumente suplimentare de cuvinte cheie pentru funcția metrică. Pentru majoritatea valorilor metrice vor fi aceleași cu parametrul metric_params, dar poate conține și valoarea parametrului p dacă atributul effective_metric_ este setat pe „minkowski”.
- outputs_2d_bool - Fals atunci când forma y este (n_samples,) sau (n_samples, 1) în timpul ajustării altfel True.

## Third model Support vector machines (SVM)
Support vector machines (SVM) sunt un set de metode de învățare supravegheate utilizate pentru clasificare, regresie și detectarea valorilor exterioare.

Implementarea se bazează pe libsvm. Timpul de încadrare se scalează cel puțin în mod quadrat cu numărul de probe și poate fi nepractic dincolo de zeci de mii de probe. Pentru seturi de date mari, luați în considerare utilizarea în schimb, sklearn.svm.LinearSVC sau sklearn.linear_model.SGDClassifier, eventual după un transformator sklearn.kernel_approximation.Nystroem.

### Avantajele mașinilor de susținere sunt:
- Eficient în spații cu dimensiuni mari.
- Încă este eficient în cazurile în care numărul de dimensiuni este mai mare decât numărul de eșantioane.
- Utilizează un subset de puncte de instruire în funcția decizională (numiți vectori de susținere), deci este, de asemenea, eficient în memorie.
- Versatil: diferite funcții Kernel pot fi specificate pentru funcția de decizie. Sunt furnizate sâmburele comune, dar este de asemenea posibil să se specifice nuclee personalizate.

### Dezavantajele mașinilor vectoriale de suport includ:
- Dacă numărul de caracteristici este mult mai mare decât numărul de eșantioane, evitați montarea excesivă în alegerea funcțiilor Kernel, iar termenul de regularizare este crucial.
- SVM-urile nu furnizează în mod direct estimări ale probabilității, acestea sunt calculate folosind o validare încrucișată de cinci ori mai scumpă (vezi Scoruri și probabilități, mai jos).



## Fourth model GaussianProcessClassifier
### Clasificarea proceselor gaussiene (GPC) bazată pe aproximarea Laplace.
Implementarea se bazează pe Algoritmul 3.1, 3.2 și 5.1 al proceselor gaussiene pentru învățarea mașinilor (GPML) de către Rasmussen și Williams.

Intern, aproximarea Laplace este utilizată pentru aproximarea posteriorului ne-gaussian de către un Gaussian.

În prezent, implementarea este restricționată la utilizarea funcției de legătură logistică. Pentru clasificarea cu mai multe clase, sunt prevăzute mai multe clasificatoare binare unu-versus. Rețineți că această clasă nu implementează astfel o adevărată aproximare Laplace cu mai multe clase.

### Parametrii:
- kernel : kernel object - Nucleul care specifică funcția de covarianță a medicului general.
- optimizer : string or callable, optional (default: “fmin_l_bfgs_b”) - Poate fi unul dintre optimizatoarele interne acceptate pentru optimizarea parametrilor nucleului, specificate de un șir, sau un optimizator definit extern transmis ca apelabil.
- n_restarts_optimizer : int, optional (default: 0) - Numărul de reporniri ale optimizatorului pentru găsirea parametrilor nucleului care maximizează probabilitatea marginală de logare.
- max_iter_predict : int, optional (default: 100) - Numărul maxim de iterații din metoda lui Newton pentru aproximarea posteriorului în timpul prezicerii.
- warm_start : bool, optional (default: False) - Dacă pornirea caldă este activată, soluția ultimei iterații Newton la aproximarea Laplace a modului posterior este utilizată ca inițializare pentru următorul apel al _posterior_mode ().
- copy_X_train : bool, optional (default: True) - Dacă este adevărat, o copie persistentă a datelor de instruire este stocată în obiect.
- random_state : int, RandomState instance or None, optional (default: None)  - Generatorul folosit pentru inițializarea centrelor. Dacă int, random_state este seed-ul folosit de generatorul de numere aleatoare; Dacă instanța RandomState, random_state este generatorul de numere aleatorii; Dacă Nici unul, generatorul de numere aleatorii este instanța RandomState folosită de np.random.
- multi_class : string, default - Specifică modul în care sunt gestionate problemele de clasificare cu mai multe clase. Sunt acceptate „one_vs_rest” și „one_vs_one”.
- n_jobs : int or None, optional (default=None) - Numărul de locuri de muncă utilizate pentru calcul.
### Atribute:
- kernel : kernel object - Nucleul folosit pentru predicție. În cazul clasificării binare, structura nucleului este aceeași cu cea trecută ca parametru, dar cu hiperparametri optimizați.
- log_marginal_likelihood_value : float - Probabilitatea log-marginal of self.kernel_.theta
- classes_ : array-like of shape (n_classes,) - Etichete de clasă unice.
- n_classes_ : int - Numărul de clase din datele de instruire

## Fifth model RandomForestClassifier
Un random forest este un meta-estimator care se potrivește cu mai multe clasificatoare de arbori de decizie pe diverse sub-eșantioane ale setului de date și folosește medie pentru a îmbunătăți exactitatea predictivă și pentru a controla supra-montarea. Mărimea sub-eșantionului este întotdeauna aceeași cu dimensiunea inițială a eșantionului de intrare, dar eșantioanele sunt trase cu înlocuire dacă bootstrap = True (implicit).

În pădurile aleatorii, fiecare copac din ansamblu este construit dintr-un eșantion extras cu înlocuire (adică, un eșantion de bootstrap) din setul de antrenament.

Mai mult, atunci când se divid, fiecare nod în timpul construcției unui arbore, cea mai bună împărțire se găsește fie din toate caracteristicile de intrare, fie dintr-un subset aleatoriu de dimensiuni max_features.

Scopul acestor două surse de aleatoriu este reducerea variației estimatorului de forest. Într-adevăr, arborii de decizie individuali prezintă, de obicei, o variație mare și tind să se împerecheze. Randomul injectat din păduri produce arbori decizionali cu erori de predicție oarecum necuplate. Luând în medie aceste predicții, unele erori pot fi anulate. Pădurile aleatorii obțin o variație redusă prin combinarea unor arbori diversi, uneori cu prețul unei ușoare creșteri a părtinirilor. În practică, reducerea variației este adesea semnificativă, rezultând deci un model global mai bun.

Valorile implicite pentru parametrii care controlează dimensiunea arborilor (de exemplu, max_depth, min_samples_leaf, etc.) duc la arbori în totalitate crescuți, care pot fi foarte mari pe unele seturi de date. Pentru a reduce consumul de memorie, complexitatea și dimensiunea arborilor ar trebui controlate aceste valori ale parametrilor.

Funcțiile sunt permise întotdeauna la întâmplare la fiecare împărțire. Prin urmare, cel mai bine divizat poate varia, chiar și cu aceleași date de instruire, max_features = n_features și bootstrap = False, dacă îmbunătățirea criteriului este identică pentru mai multe divizii enumerate în timpul căutării celei mai bune împărțiri. Pentru a obține un comportament determinist în timpul montării, random_state trebuie să fie stabilit.

### Exercitiul 4 (numar de modele * numar de seturi de date \* 1 punct = 20 de puncte)
Pentru fiecare model: efectuati o cautare a hiperparametrilor optimi folosind grid search si random search (cu parametrul cv = 4), folosind 5 fold cross validation.

In [28]:
parameters_dtc = {'splitter': ['best', 'random'], 'max_depth': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
parameters_knn = {'n_neighbors': list(range(1, 10)), 'p': [1, 2, 3, 4.7]}
parameters_svc = {'gamma': ['auto', 'scale'], 'C': list(range(1, 10))}
parameters_gpc = {'random_state': [0, None]}
parameters_rfc = {'max_depth': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}

# First dataset 'audit'
## First model used: DecisionTreeClassifier
### GridSearchCV and RandomizedSearchCV

In [29]:
grid_search_dtc_audit = GridSearchCV(estimator=dtc_audit, param_grid=parameters_dtc, scoring='accuracy', cv=4)
grid_search_dtc_audit.fit(X_audit, y_audit)
grid_scores_dtc_audit = cross_val_score(grid_search_dtc_audit, X_audit, y_audit, cv=5)
print(grid_search_dtc_audit.best_params_)
print(grid_scores_dtc_audit.mean())

randomized_search_dtc_audit = RandomizedSearchCV(estimator=dtc_audit, param_distributions=parameters_dtc, random_state=0, scoring='accuracy', cv=4)
randomized_search_dtc_audit.fit(X_audit, y_audit)
randomized_scores_dtc_audit = cross_val_score(randomized_search_dtc_audit, X_audit, y_audit, cv=5)
print(randomized_search_dtc_audit.best_params_)
print(randomized_scores_dtc_audit.mean())

{'max_depth': 1.0, 'splitter': 'best'}
1.0
{'splitter': 'best', 'max_depth': 1.0}
1.0


# First dataset 'audit'
## Second model used: KNeighborsClassifier
### GridSearchCV and RandomizedSearchCV

In [30]:
grid_search_knn_audit = GridSearchCV(estimator=knn_audit, param_grid=parameters_knn, scoring='accuracy', cv=4)
grid_search_knn_audit.fit(X_audit, y_audit)
grid_scores_knn_audit = cross_val_score(grid_search_knn_audit, X_audit, y_audit, cv=5)
print(grid_search_knn_audit.best_params_)
print(grid_scores_knn_audit.mean())

randomized_search_knn_audit = RandomizedSearchCV(estimator=knn_audit, param_distributions=parameters_knn, random_state=0, scoring='accuracy', cv=4)
randomized_search_knn_audit.fit(X_audit, y_audit)
randomized_scores_knn_audit = cross_val_score(randomized_search_knn_audit, X_audit, y_audit, cv=5)
print(randomized_search_knn_audit.best_params_)
print(randomized_scores_knn_audit.mean())

{'n_neighbors': 9, 'p': 1}
0.8374441687344912
{'p': 3, 'n_neighbors': 1}
0.8309925558312654


# First dataset 'audit'
## Third model used: SVM.SVC
### GridSearchCV and RandomizedSearchCV

In [31]:
grid_search_svc_audit = GridSearchCV(estimator=svc_audit, param_grid=parameters_svc, scoring='accuracy', cv=4)
grid_search_svc_audit.fit(X_audit, y_audit)
grid_scores_svc_audit = cross_val_score(grid_search_svc_audit, X_audit, y_audit, cv=5)
print(grid_search_svc_audit.best_params_)
print(grid_scores_svc_audit.mean())

randomized_search_svc_audit = RandomizedSearchCV(estimator=svc_audit, param_distributions=parameters_svc, random_state=0, scoring='accuracy', cv=4)
randomized_search_svc_audit.fit(X_audit, y_audit)
randomized_scores_svc_audit = cross_val_score(randomized_search_svc_audit, X_audit, y_audit, cv=5)
print(randomized_search_svc_audit.best_params_)
print(randomized_scores_svc_audit.mean())

{'C': 3, 'gamma': 'auto'}
0.8168072787427626
{'gamma': 'auto', 'C': 4}
0.8219685690653433


# First dataset 'audit'
## Fourth model used: GaussianProcessClassifier
### GridSearchCV and RandomizedSearchCV

In [32]:
grid_search_gpc_audit = GridSearchCV(estimator=gpc_audit, param_grid=parameters_gpc, scoring='accuracy', cv=4)
grid_search_gpc_audit.fit(X_audit, y_audit)
grid_scores_gpc_audit = cross_val_score(grid_search_gpc_audit, X_audit, y_audit, cv=5)
print(grid_search_gpc_audit.best_params_)
print(grid_scores_gpc_audit.mean())

randomized_search_gpc_audit = RandomizedSearchCV(estimator=gpc_audit, param_distributions=parameters_gpc, random_state=0, scoring='accuracy', cv=4)
randomized_search_gpc_audit.fit(X_audit, y_audit)
randomized_scores_gpc_audit = cross_val_score(randomized_search_gpc_audit, X_audit, y_audit, cv=5)
print(randomized_search_gpc_audit.best_params_)
print(randomized_scores_gpc_audit.mean())

{'random_state': 0}
0.8129611248966088
{'random_state': 0}
0.8129611248966088


# First dataset 'audit'
## Fifth model used: RandomForestClassifier
### GridSearchCV and RandomizedSearchCV

In [33]:
grid_search_rfc_audit = GridSearchCV(estimator=rcf_audit, param_grid=parameters_rfc, scoring='accuracy', cv=4)
grid_search_rfc_audit.fit(X_audit, y_audit)
grid_scores_rfc_audit = cross_val_score(grid_search_rfc_audit, X_audit, y_audit, cv=5)
print(grid_search_rfc_audit.best_params_)
print(grid_scores_rfc_audit.mean())

randomized_search_rfc_audit = RandomizedSearchCV(estimator=rcf_audit, param_distributions=parameters_rfc, random_state=0, scoring='accuracy', cv=4)
randomized_search_rfc_audit.fit(X_audit, y_audit)
randomized_scores_rfc_audit = cross_val_score(randomized_search_rfc_audit, X_audit, y_audit, cv=5)
print(randomized_search_rfc_audit.best_params_)
print(randomized_scores_rfc_audit.mean())

{'max_depth': 1.0}
0.9729114971050455
{'max_depth': 1.0}
0.9729114971050455


# Second dataset 'banknote'
## First model used: DecisionTreeClassifier
### GridSearchCV and RandomizedSearchCV

In [34]:
grid_search_dtc_bank = GridSearchCV(estimator=dtc_bank, param_grid=parameters_dtc, scoring='accuracy', cv=4)
grid_search_dtc_bank.fit(X_bank, y_bank)
grid_scores_dtc_bank = cross_val_score(grid_search_dtc_bank, X_bank, y_bank, cv=5)
print(grid_search_dtc_bank.best_params_)
print(grid_scores_dtc_bank.mean())

randomized_search_dtc_bank = RandomizedSearchCV(estimator=dtc_bank, param_distributions=parameters_dtc, random_state=0, scoring='accuracy', cv=4)
randomized_search_dtc_bank.fit(X_bank, y_bank)
randomized_scores_dtc_bank = cross_val_score(randomized_search_dtc_bank, X_bank, y_bank, cv=5)
print(randomized_search_dtc_bank.best_params_)
print(randomized_scores_dtc_bank.mean())

{'max_depth': 1.0, 'splitter': 'best'}
0.8527856668878566
{'splitter': 'best', 'max_depth': 1.0}
0.8527856668878566


# Second dataset 'banknote'
## Second model used: KNeighborsClassifier
### GridSearchCV and RandomizedSearchCV

In [35]:
grid_search_knn_bank = GridSearchCV(estimator=knn_bank, param_grid=parameters_knn, scoring='accuracy', cv=4)
grid_search_knn_bank.fit(X_bank, y_bank)
grid_scores_knn_bank = cross_val_score(grid_search_knn_bank, X_bank, y_bank, cv=5)
print(grid_search_knn_bank.best_params_)
print(grid_scores_knn_bank.mean())

randomized_search_knn_bank = RandomizedSearchCV(estimator=knn_bank, param_distributions=parameters_knn, random_state=0, scoring='accuracy', cv=4)
randomized_search_knn_bank.fit(X_bank, y_bank)
randomized_scores_knn_bank = cross_val_score(randomized_search_knn_bank, X_bank, y_bank, cv=5)
print(randomized_search_knn_bank.best_params_)
print(randomized_scores_knn_bank.mean())

{'n_neighbors': 7, 'p': 1}
0.9963530192435301
{'p': 1, 'n_neighbors': 6}
0.9956230922362309


# Second dataset 'banknote'
## Third model used: SVM.SVC
### GridSearchCV and RandomizedSearchCV

In [36]:
grid_search_svc_bank = GridSearchCV(estimator=svc_bank, param_grid=parameters_svc, scoring='accuracy', cv=4)
grid_search_svc_bank.fit(X_bank, y_bank)
grid_scores_svc_bank = cross_val_score(grid_search_svc_bank, X_bank, y_bank, cv=5)
print(grid_search_svc_bank.best_params_)
print(grid_scores_svc_bank.mean())

randomized_search_svc_bank = RandomizedSearchCV(estimator=svc_bank, param_distributions=parameters_svc, random_state=0, scoring='accuracy', cv=4)
randomized_search_svc_bank.fit(X_bank, y_bank)
randomized_scores_svc_bank = cross_val_score(randomized_search_svc_bank, X_bank, y_bank, cv=5)
print(randomized_search_svc_bank.best_params_)
print(randomized_scores_svc_bank.mean())

{'C': 1, 'gamma': 'auto'}
0.9978128732581286
{'gamma': 'auto', 'C': 4}
0.9978128732581286


# Second dataset 'banknote'
## Fourth model used: GaussianProcessClassifier
### GridSearchCV and RandomizedSearchCV

In [37]:
grid_search_gpc_bank = GridSearchCV(estimator=gpc_bank, param_grid=parameters_gpc, scoring='accuracy', cv=4)
grid_search_gpc_bank.fit(X_bank, y_bank)
grid_scores_gpc_bank = cross_val_score(grid_search_gpc_bank, X_bank, y_bank, cv=5)
print(grid_search_gpc_bank.best_params_)
print(grid_scores_gpc_bank.mean())

randomized_search_gpc_bank = RandomizedSearchCV(estimator=gpc_bank, param_distributions=parameters_gpc, random_state=0, scoring='accuracy', cv=4)
randomized_search_gpc_bank.fit(X_bank, y_bank)
randomized_scores_gpc_bank = cross_val_score(randomized_search_gpc_bank, X_bank, y_bank, cv=5)
print(randomized_search_gpc_bank.best_params_)
print(randomized_scores_gpc_bank.mean())

{'random_state': 0}
0.9970829462508295
{'random_state': 0}
0.9970829462508295


# Second dataset 'banknote'
## First model used: RandomForestClassifier
### GridSearchCV and RandomizedSearchCV

In [38]:
grid_search_rfc_bank = GridSearchCV(estimator=rcf_bank, param_grid=parameters_rfc, scoring='accuracy', cv=4)
grid_search_rfc_bank.fit(X_bank, y_bank)
grid_scores_rfc_bank = cross_val_score(grid_search_rfc_bank, X_bank, y_bank, cv=5)
print(grid_search_rfc_bank.best_params_)
print(grid_scores_rfc_bank.mean())

randomized_search_rfc_bank = RandomizedSearchCV(estimator=rcf_bank, param_distributions=parameters_rfc, random_state=0, scoring='accuracy', cv=4)
randomized_search_rfc_bank.fit(X_bank, y_bank)
randomized_scores_rfc_bank = cross_val_score(randomized_search_rfc_bank, X_bank, y_bank, cv=5)
print(randomized_search_rfc_bank.best_params_)
print(randomized_scores_rfc_bank.mean())

{'max_depth': 1.0}
0.8935952222959521
{'max_depth': 1.0}
0.8935952222959521


# Third dataset 'breastCancer'
## First model used: DecisionTreeClassifier
### GridSearchCV and RandomizedSearchCV

In [39]:
grid_search_dtc_cancer = GridSearchCV(estimator=dtc_cancer, param_grid=parameters_dtc, scoring='accuracy', cv=4)
grid_search_dtc_cancer.fit(X_cancer, y_cancer)
grid_scores_dtc_cancer = cross_val_score(grid_search_dtc_cancer, X_cancer, y_cancer, cv=5)
print(grid_search_dtc_cancer.best_params_)
print(grid_scores_dtc_cancer.mean())

randomized_search_dtc_cancer = RandomizedSearchCV(estimator=dtc_cancer, param_distributions=parameters_dtc, random_state=0, scoring='accuracy', cv=4)
randomized_search_dtc_cancer.fit(X_cancer, y_cancer)
randomized_scores_dtc_cancer = cross_val_score(randomized_search_dtc_cancer, X_cancer, y_cancer, cv=5)
print(randomized_search_dtc_cancer.best_params_)
print(randomized_scores_dtc_cancer.mean())

{'max_depth': 1.0, 'splitter': 'best'}
0.6384057971014492
{'splitter': 'best', 'max_depth': 1.0}
0.6384057971014492


# Third dataset 'breastCancer'
## Second model used: KNeighborsClassifier
### GridSearchCV and RandomizedSearchCV

In [40]:
grid_search_knn_cancer = GridSearchCV(estimator=knn_cancer, param_grid=parameters_knn, scoring='accuracy', cv=4)
grid_search_knn_cancer.fit(X_cancer, y_cancer)
grid_scores_knn_cancer = cross_val_score(grid_search_knn_cancer, X_cancer, y_cancer, cv=5)
print(grid_search_knn_cancer.best_params_)
print(grid_scores_knn_cancer.mean())

randomized_search_knn_cancer = RandomizedSearchCV(estimator=knn_cancer, param_distributions=parameters_knn, random_state=0, scoring='accuracy', cv=4)
randomized_search_knn_cancer.fit(X_cancer, y_cancer)
randomized_scores_knn_cancer = cross_val_score(randomized_search_knn_cancer, X_cancer, y_cancer, cv=5)
print(randomized_search_knn_cancer.best_params_)
print(randomized_scores_knn_cancer.mean())

{'n_neighbors': 1, 'p': 1}
0.5427536231884058
{'p': 3, 'n_neighbors': 1}
0.5521739130434782


# Third dataset 'breastCancer'
## Third model used: SVM.SVC
### GridSearchCV and RandomizedSearchCV

In [41]:
grid_search_svc_cancer = GridSearchCV(estimator=svc_cancer, param_grid=parameters_svc, scoring='accuracy', cv=4)
grid_search_svc_cancer.fit(X_cancer, y_cancer)
grid_scores_svc_cancer = cross_val_score(grid_search_svc_cancer, X_cancer, y_cancer, cv=5)
print(grid_search_svc_cancer.best_params_)
print(grid_scores_svc_cancer.mean())

randomized_search_svc_cancer = RandomizedSearchCV(estimator=svc_cancer, param_distributions=parameters_svc, random_state=0, scoring='accuracy', cv=4)
randomized_search_svc_cancer.fit(X_cancer, y_cancer)
randomized_scores_svc_cancer = cross_val_score(randomized_search_svc_cancer, X_cancer, y_cancer, cv=5)
print(randomized_search_svc_cancer.best_params_)
print(randomized_scores_svc_cancer.mean())

{'C': 8, 'gamma': 'scale'}
0.5692028985507246
{'gamma': 'scale', 'C': 9}
0.5692028985507246


# Third dataset 'breastCancer'
## Fourth model used: GaussianProcessClassifier
### GridSearchCV and RandomizedSearchCV

In [42]:
grid_search_gpc_cancer = GridSearchCV(estimator=gpc_cancer, param_grid=parameters_gpc, scoring='accuracy', cv=4)
grid_search_gpc_cancer.fit(X_cancer, y_cancer)
grid_scores_gpc_cancer = cross_val_score(grid_search_gpc_cancer, X_cancer, y_cancer, cv=5)
print(grid_search_gpc_cancer.best_params_)
print(grid_scores_gpc_cancer.mean())

randomized_search_gpc_cancer = RandomizedSearchCV(estimator=gpc_cancer, param_distributions=parameters_gpc, random_state=0, scoring='accuracy', cv=4)
randomized_search_gpc_cancer.fit(X_cancer, y_cancer)
randomized_scores_gpc_cancer = cross_val_score(randomized_search_gpc_audit, X_cancer, y_cancer, cv=5)
print(randomized_search_gpc_audit.best_params_)
print(randomized_scores_gpc_audit.mean())

{'random_state': 0}
0.46449275362318837
{'random_state': 0}
0.8129611248966088


# Third dataset 'breastCancer'
## Fifth model used: RandomForestClassifier
### GridSearchCV and RandomizedSearchCV

In [43]:
grid_search_rfc_cancer = GridSearchCV(estimator=rcf_cancer, param_grid=parameters_rfc, scoring='accuracy', cv=4)
grid_search_rfc_cancer.fit(X_cancer, y_cancer)
grid_scores_rfc_cancer = cross_val_score(grid_search_rfc_cancer, X_cancer, y_cancer, cv=5)
print(grid_search_rfc_cancer.best_params_)
print(grid_scores_rfc_cancer.mean())

randomized_search_rfc_cancer = RandomizedSearchCV(estimator=rcf_cancer, param_distributions=parameters_rfc, random_state=0, scoring='accuracy', cv=4)
randomized_search_rfc_cancer.fit(X_cancer, y_cancer)
randomized_scores_rfc_cancer = cross_val_score(randomized_search_rfc_cancer, X_cancer, y_cancer, cv=5)
print(randomized_search_rfc_cancer.best_params_)
print(randomized_scores_rfc_cancer.mean())

{'max_depth': 1.0}
0.6550724637681159
{'max_depth': 1.0}
0.6550724637681159


# Fourth dataset 'ionosphere'
## First model used: DecisionTreeClassifier
### GridSearchCV and RandomizedSearchCV

In [44]:
grid_search_dtc_ionosphere = GridSearchCV(estimator=dtc_ionosphere, param_grid=parameters_dtc, scoring='accuracy', cv=4)
grid_search_dtc_ionosphere.fit(X_ionosphere, y_ionosphere)
grid_scores_dtc_ionosphere = cross_val_score(grid_search_dtc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(grid_search_dtc_ionosphere.best_params_)
print(grid_scores_dtc_ionosphere.mean())

randomized_search_dtc_ionosphere = RandomizedSearchCV(estimator=dtc_ionosphere, param_distributions=parameters_dtc, random_state=0, scoring='accuracy', cv=4)
randomized_search_dtc_ionosphere.fit(X_ionosphere, y_ionosphere)
randomized_scores_dtc_ionosphere = cross_val_score(randomized_search_dtc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(randomized_search_dtc_ionosphere.best_params_)
print(randomized_scores_dtc_ionosphere.mean())

{'max_depth': 1.0, 'splitter': 'random'}
0.829215291750503
{'splitter': 'random', 'max_depth': 1.0}
0.829215291750503


# Fourth dataset 'ionosphere'
## Second model used: KNeighborsClassifier
### GridSearchCV and RandomizedSearchCV

In [45]:
grid_search_knn_ionosphere = GridSearchCV(estimator=knn_ionosphere, param_grid=parameters_knn, scoring='accuracy', cv=4)
grid_search_knn_ionosphere.fit(X_ionosphere, y_ionosphere)
grid_scores_knn_ionosphere = cross_val_score(grid_search_knn_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(grid_search_knn_ionosphere.best_params_)
print(grid_scores_knn_ionosphere.mean())

randomized_search_knn_ionosphere = RandomizedSearchCV(estimator=knn_ionosphere, param_distributions=parameters_knn, random_state=0, scoring='accuracy', cv=4)
randomized_search_knn_ionosphere.fit(X_ionosphere, y_ionosphere)
randomized_scores_knn_ionosphere = cross_val_score(randomized_search_knn_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(randomized_search_knn_ionosphere.best_params_)
print(randomized_scores_knn_ionosphere.mean())

{'n_neighbors': 2, 'p': 1}
0.8974647887323945
{'p': 1, 'n_neighbors': 6}
0.8917505030181087


# Fourth dataset 'ionosphere'
## Third model used: SVM.SVC
### GridSearchCV and RandomizedSearchCV

In [46]:
grid_search_svc_ionosphere = GridSearchCV(estimator=svc_ionosphere, param_grid=parameters_svc, scoring='accuracy', cv=4)
grid_search_svc_ionosphere.fit(X_ionosphere, y_ionosphere)
grid_scores_svc_ionosphere = cross_val_score(grid_search_svc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(grid_search_svc_ionosphere.best_params_)
print(grid_scores_svc_ionosphere.mean())

randomized_search_svc_ionosphere = RandomizedSearchCV(estimator=svc_ionosphere, param_distributions=parameters_svc, random_state=0, scoring='accuracy', cv=4)
randomized_search_svc_ionosphere.fit(X_ionosphere, y_ionosphere)
randomized_scores_svc_ionosphere = cross_val_score(randomized_search_svc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(randomized_search_svc_ionosphere.best_params_)
print(randomized_scores_svc_ionosphere.mean())

{'C': 4, 'gamma': 'scale'}
0.9487323943661972
{'gamma': 'scale', 'C': 9}
0.948692152917505


# Fourth dataset 'ionosphere'
## Fourth model used: GaussianProcessClassifier
### GridSearchCV and RandomizedSearchCV

In [47]:
grid_search_gpc_ionosphere = GridSearchCV(estimator=gpc_ionosphere, param_grid=parameters_gpc, scoring='accuracy', cv=4)
grid_search_gpc_ionosphere.fit(X_ionosphere, y_ionosphere)
grid_scores_gpc_ionosphere = cross_val_score(grid_search_gpc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(grid_search_gpc_ionosphere.best_params_)
print(grid_scores_gpc_ionosphere.mean())

randomized_search_gpc_ionosphere = RandomizedSearchCV(estimator=gpc_ionosphere, param_distributions=parameters_gpc, random_state=0, scoring='accuracy', cv=4)
randomized_search_gpc_ionosphere.fit(X_ionosphere, y_ionosphere)
randomized_scores_gpc_ionosphere = cross_val_score(randomized_search_gpc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(randomized_search_gpc_ionosphere.best_params_)
print(randomized_scores_gpc_ionosphere.mean())

{'random_state': 0}
0.8490140845070424
{'random_state': 0}
0.8490140845070424


# Fourth dataset 'ionosphere'
## Fifth model used: RandomForestClassifier
### GridSearchCV and RandomizedSearchCV

In [48]:
grid_search_rfc_ionosphere = GridSearchCV(estimator=rcf_ionosphere, param_grid=parameters_rfc, scoring='accuracy', cv=4)
grid_search_rfc_ionosphere.fit(X_ionosphere, y_ionosphere)
grid_scores_rfc_ionosphere = cross_val_score(grid_search_rfc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(grid_search_rfc_ionosphere.best_params_)
print(grid_scores_rfc_ionosphere.mean())

randomized_search_rfc_ionosphere = RandomizedSearchCV(estimator=rcf_ionosphere, param_distributions=parameters_rfc, random_state=0, scoring='accuracy', cv=4)
randomized_search_rfc_ionosphere.fit(X_ionosphere, y_ionosphere)
randomized_scores_rfc_ionosphere = cross_val_score(randomized_search_rfc_ionosphere, X_ionosphere, y_ionosphere, cv=5)
print(randomized_search_rfc_ionosphere.best_params_)
print(randomized_scores_rfc_ionosphere.mean())

{'max_depth': 1.0}
0.8490543259557344
{'max_depth': 1.0}
0.8490543259557344
